In [1]:
%%capture
! pip install gensim

In [2]:
from gensim.models import doc2vec
import pandas as pd
from collections import namedtuple
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize
from time import time
import re
import numpy as np

In [3]:
data = pd.read_csv('./Books.csv', encoding = 'utf-8')

In [4]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    size=100,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=8,   # multi cpu
    hs = 1,          # hierar chical softmax / default 0
    negative = 10   # negative sampling / default 5
)

/usr/local/lib/python3.8/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [5]:
agg = data[['id', 'title', 'overview']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['title', 'overview']].values]

In [6]:
doc_vectorizer.build_vocab(tagged_train_docs)

In [7]:
start = time()

for epoch in range(5):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

#doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
end = time()
print("During Time: {}".format(end-start))

<ipython-input-7-dd0958075b5c>:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)


During Time: 496.4702091217041


In [8]:
doc_vectorizer.docvecs.most_similar('내가 틀릴 수도 있습니다 - 숲속의 현자가 전하는 마지막 인생 수업', topn = 10 )

[('나는 나무처럼 살고 싶다 - 30년간 아픈 나무들을 돌봐 온 나무 의사 우종영이 나무에게 배운 단단한 삶의 지혜 35',
  0.4319668114185333),
 ('몰입의 즐거움 - [ 양장 ]', 0.4260326623916626),
 ('어른 그림책 여행 - 내 마음을 둘러보고 싶을 때', 0.4240935444831848),
 ('삶을 여행하는 초심자를 위한 - 죽음 가이드북', 0.4179852604866028),
 ('마음의 요가 - 인도 최고의 지성과 영성, 비베카난다의 말', 0.3797704875469208),
 ('책의 맛 - 로제 그르니에가 펼쳐 보이는 문학의 세계', 0.37742409110069275),
 ('아비투스 - 인간의 품격을 결정하는 7가지 자본', 0.37696927785873413),
 ('길 위의 철학자 - 떠돌이 철학자의 삶에 관한 에피소드', 0.3743991255760193),
 ('오후의 글쓰기 - 자발적 글쓰기를 시작하는 어른을 위한 따뜻한 문장들', 0.36943519115448),
 ('모든 맛에는 이유가 있다 - 인문학에 과학으로 감칠맛을 더한 가장 지적인 파인다이닝', 0.3643539845943451)]